In [1]:
from typing import Union
from tqdm import tqdm

import pandas as pd
import numpy as np

from scipy import stats
from statsmodels.stats.meta_analysis import effectsize_smd
from statsmodels.stats import proportion
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.power import zt_ind_solve_power

import plotly.express as px

# Задание
**На сайте запущен А/В тест с целью увеличить доход. В приложенном excel файле вы найдете сырые данные по результатам эксперимента – user_id, тип выборки variant_name и доход принесенный пользователем revenue.**

**Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.**

Считаем данные из файла

In [4]:
data = pd.read_excel('D:\Катя\GeekBrains\! 21. AB-тестирование\Семинар 8 Python применение статистических критериев на практике - пайплайн оценки AB/gb_sem_8_hm.xlsx')

data.head(10)

C:\Users\Daxa\anaconda3\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.0
1,2423,control,0.0
2,9411,control,0.0
3,7311,control,0.0
4,6174,variant,0.0
5,2380,variant,0.0
6,2849,control,0.0
7,9168,control,0.0
8,6205,variant,0.0
9,7548,control,0.0


Общая информация о датасете

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   USER_ID       10000 non-null  int64  
 1   VARIANT_NAME  10000 non-null  object 
 2   REVENUE       10000 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 234.5+ KB


Датасет содержит 10 000 строк и 3 столбца с данными. Столбец USER_ID содержит целочисленные значения, столбец VARIANT_NAME - строковые, столбец REVENUE - вещественные значения. Пустых ячеек нет.

Перед применением статистических критериев для оценки результатов теста выполним качественную проверку данных в датасете. 

Для начала посмотрим, есть ли не уникальные значения id юзеров. Иными словами посмотрим, не мог ли один юзер попасть в обе группы при проведении эксперимента или попасть в одну группу несколько раз.

In [13]:
data.USER_ID.nunique()

6324

Видим, что в данных достаточно много не уникальных id - больше половины. Это уже может говорить либо о некорректно работающем сплитовании или же о некорректно проведенном тесте. Чтобы сделать окончательный вывод продолжем обработку данных.

Просуммируем значения REVENUE для каждого не уникального id по каждой из групп.

In [15]:
data = data.groupby(['USER_ID', 'VARIANT_NAME'], as_index=False).agg({'REVENUE': 'sum'})
data.shape

(7865, 3)

Видим, что размер датасета сократился на более чем 2 000 строк.

Теперь уберем из данных тех пользователей, которые попали в обе группы и создадим новый датасет, очищенный от некорректных данных, для дальнейшего его анализа.

In [16]:
data.groupby('USER_ID', as_index=False).agg({'VARIANT_NAME': 'count'})['VARIANT_NAME'].value_counts()

1    4783
2    1541
Name: VARIANT_NAME, dtype: int64

In [17]:
unique_ids = \
(data
 .groupby('USER_ID', as_index=False)
 .agg({'VARIANT_NAME': 'count'})
 #.['VARIANT_NAME'].value_counts()
 .query('VARIANT_NAME == 1')
 .USER_ID
 .values
 )

In [18]:
df_new = data[data.USER_ID.isin(unique_ids)].copy(deep=True)

Посмотрим информацию по новому датасету

In [20]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4783 entries, 0 to 7862
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   USER_ID       4783 non-null   int64  
 1   VARIANT_NAME  4783 non-null   object 
 2   REVENUE       4783 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 149.5+ KB


Посмотрим статистики по каждой из групп

In [24]:
control = df_new[df_new['VARIANT_NAME'] == 'control'].copy(deep=True)
test = df_new[df_new['VARIANT_NAME'] == 'variant'].copy(deep=True)

In [25]:
control.describe()

,USER_ID,REVENUE
count,2390.000000,2390.000000
mean,5020.881590,0.196887
std,2904.850992,4.172201
min,2.000000,0.000000
25%,2517.250000,0.000000
50%,5012.500000,0.000000
75%,7616.000000,0.000000
max,9998.000000,196.010000


In [26]:
test.describe()

,USER_ID,REVENUE
count,2393.000000,2393.000000
mean,4967.943168,0.074935
std,2892.745368,0.858207
min,4.000000,0.000000
25%,2435.000000,0.000000
50%,4955.000000,0.000000
75%,7379.000000,0.000000
max,9995.000000,23.040000


Видим, что контрольная и тестовая группа практически идентичны по количеству юзеров, но при этом средняя выручка в контрольной группе почти в 3 раза выше.

**Применим статистичечкие критерии для анализа данных эксперимента**

Выручка - величина непрерывная, поэтому создадим функцию, результатом выполнеия которой будет вывод основных статистических критериев по подготовленным результатам эксперимента.

In [27]:
def continious_result(control: pd.DataFrame, # данные по контрольной группе
                      treatment: pd.DataFrame, # данные по тестовой группе
                      column: str, # колонка, по которой собираемся производить расчеты эффекта
                      n_iters: int = 10_000) -> pd.DataFrame: #количество итераций
    # Статистика по выборкам
    size = control.loc[:, column].shape[0] # выбираем размер
    
    control_mean = control.loc[:, column].mean() # считаем среднее по контролю
    treatment_mean = treatment.loc[:, column].mean() # считаем среднее по тесту
    
    control_std = control.loc[:, column].std(ddof=1) # стандартное отклонение по контролю
    treatment_std = treatment.loc[:, column].std(ddof=1) # стандартное отклонение по тесту
    
    # Бутсрап
    booted_diff = []
    for _ in tqdm(range(n_iters)):
        control_sample = control.loc[:, column].sample(n=size, replace=True).values
        treatment_sample = treatment.loc[:, column].sample(n=size, replace=True).values
        booted_diff.append(np.mean(control_sample - treatment_sample))
    
    # Считаем статистику после бустрапа
    md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1) # разницы средних, стандартных отклонений
    left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5]) # левая и правая границы доверительного интервала
    p_value_ci = 2 * (1 - stats.norm.cdf(np.abs(md_ci / std_ci))) # pvalue
    
    # Считаем мощность эксперимента
    effect_size, _ = effectsize_smd(mean1=treatment_mean, sd1=treatment_std, nobs1=size,
                                    mean2=control_mean, sd2=control_std, nobs2=size)
    power = tt_ind_solve_power(effect_size=effect_size,
                               nobs1=size,
                               alpha=.05,
                               power=None,
                               ratio=1)
    # Формируем отчёт 
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha': p_value_ci, 
                           'beta': (1-power),
                           'CI': f'[{np.round(left_ci, 3)}, {np.round(right_ci, 3)}]',
                           'difference': md_ci,},
                          index=[column]) 
    return result

Но в начале сделаем визуальный анализ полученных данных.

In [29]:
fig = px.histogram(data, x="REVENUE",
                   color='VARIANT_NAME', barmode='group',
                   height=400)
fig.show()

1. Разреженная метрика, большей частью собранная в районе нуля
2. Есть "тяжелые" хвосты
3. Распределение отличное от Бернулли

Считаем статистику

In [30]:
continious_result(control, test, column='REVENUE')

100%|██████████| 10000/10000 [00:04<00:00, 2256.22it/s]


,effect_size,alpha,beta,CI,difference
REVENUE,-0.040483,0.15813,0.712143,"[-0.005, 0.316]",0.12201


Завышенные значения alpha и beta и 0 в доверительном интервале говорят о том, что между группами нет статистически значимой разницы.

## Заключение
Качественная проверка данных эксперимента показала:
* наличие большого числа не уникальных id (больше половины);
* большого числа юзеров, попавших одновременно и в контрольную и в тестовую группы
* качественные данные составляют менее 50% от всего объема данных по эксперимену

Статистический анализ данных показал:
* мощность теста по пригодным для исследования данным очень низка
* исследование на основе статистических критериев не выявило разницу между контрольной и тестовой группой (alpha > 0,05, 0 входит в доверительный интервал)

### Рекомендации
* пересмотреть систему сплитования
* провести А/А тест для оценки эффективности новой системы сплитования
* провести повторный А/В тест